<a href="https://colab.research.google.com/github/TurgutOzkan/DataSciencity/blob/master/SQL_Window_Functions_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This post will focus on learning the basics of SQL window functions, which will help you solve challenging queries, especially in data science interviews.  Actually, they are very useful once you learn how to use but initially most people kind of feel distant from the idea of using window functions, for some reason. We will use the data we created in a previous [post](https://datasciencity.com/2020/12/04/how-to-use-sqlite-database-on-google-colab/). If you want to follow along, please generate the database as shown in that post. Window functions perform operations on predefined rows without aggregating, like the GROUP BY function. Therefore, rows do not need to be reduced to a single value. Think like getting some form of calculation for each row. In general, a typical window function looks like below:

SELECT column1, 
      SUM() OVER ( )
FROM table

There are alternatives to SUM() based on the kind of process you need, and there are things that can be specified in the OVER() clause. So basically, these are two main components in a window function. Window functions allow us to arrange rows within each partition and this is where we specify within the OVER() clause. To sort rows, we modify the OVER clause with the ORDER BY clause. Since the default version of SQLite in Colab as of this writing is 3.22, but we need 3.25+ to use window functions. For this reason, we first install the newer version as below, then restart the runtime, check the version, and lastly run the code in the previous post to populate a database.

In [2]:
# window function is supported in 3.25+ so we have to upgrade it then restart the runtime so that we can work on window functions
!add-apt-repository -y ppa:sergey-dryabzhinsky/packages
!apt update
!apt install sqlite3

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Get:12 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:13 http://ppa.launchpad.net/sergey-dryabzhinsky/packages/ubuntu bi

In [18]:
# Now, restart the runtime (not the factory restart!). Then run below to see if the new version is no longer 3.22.
import sqlite3
print(sqlite3.sqlite_version)

3.33.0


We confirmed that our SQLite version is correct. Let's check the data. To provide database-like visual qualities, I will use pandas data frames below so that you can easily follow along with the results of each query.

In [43]:
cursor = conn.execute(''' SELECT * FROM Employees ''')
 
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['ssn', 'name', 'lastName', 'Department', 'Salary'])

,ssn,name,lastName,Department,Salary
0,123234877,Michael,Rogers,14,100
1,152934485,Anand,Manikutty,14,300
2,222364883,Carol,Smith,37,200
3,326587417,Joe,Stevens,37,150
4,332154719,Mary-Anne,Foster,14,500
5,332569843,George,O'Donnell,77,140
6,546523478,John,Doe,59,130
7,631231482,David,Smith,77,90
8,654873219,Zacary,Efron,59,600
9,745685214,Eric,Goldsmith,59,130


We have SSN, Name, LastName, Department, and Salary columns in the Employees table. Let's run some window functions from the most basic to a little more complex ones so we see what's happening under the hood. First, see what happens when leaving OVER() as it is without specifying anything in it.

In [46]:
cursor = conn.execute(''' SELECT name, Department, Salary,
 SUM(Salary) OVER() as SUM_Salary FROM Employees''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'SUM_Salary'])

,name,Department,Salary,SUM_Salary
0,Michael,14,100,2440
1,Anand,14,300,2440
2,Carol,37,200,2440
3,Joe,37,150,2440
4,Mary-Anne,14,500,2440
5,George,77,140,2440
6,John,59,130,2440
7,David,77,90,2440
8,Zacary,59,600,2440
9,Eric,59,130,2440


So we got all rows back with each row assigned to the total salary 2440. Seems like SUM() is applied to the entire data. So OVER() is like OVER (ENTIRE DATA)  in this case when left unspecified. Good for a start but not super useful. Now, let's find each person's group salary sum. For example, working in IT, what is my groups' total salary? We want to 'partition' folks by their department. 

- PARTITION BY defines the area to apply a window function. Therefore, by saying SUM(Salary) OVER(PARTITION BY Department), we say apply summing operation but do that for each department separately and give me the total sum for each department.

Let's apply partitioning in the OVER() clause.


In [22]:
cursor = conn.execute(''' SELECT name, Department, Salary, SUM(Salary) OVER(PARTITION BY Department) as SUM_Salary FROM Employees''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'SUM_Salary'])

,name,Department,Salary,SUM_Salary
0,Michael,14,100,1000
1,Anand,14,300,1000
2,Mary-Anne,14,500,1000
3,Elizabeth,14,50,1000
4,Kumar,14,50,1000
5,Carol,37,200,350
6,Joe,37,150,350
7,John,59,130,860
8,Zacary,59,600,860
9,Eric,59,130,860


Better! So department 14 brings in 1000 in total, and each row is assigned to its corresponding department salary sum. Let's see what happens when we order by salary as we usually use the ORDER BY statement in non-window function type queries. So we put it at the end.

In [44]:
cursor = conn.execute(''' SELECT name, Department, Salary, SUM(Salary) OVER(PARTITION BY Department)
 as SUM_Salary FROM Employees Order BY Salary DESC''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'SUM_Salary'])

,name,Department,Salary,SUM_Salary
0,Zacary,59,600,860
1,Mary-Anne,14,500,1000
2,Anand,14,300,1000
3,Carol,37,200,350
4,Joe,37,150,350
5,George,77,140,230
6,John,59,130,860
7,Eric,59,130,860
8,Michael,14,100,1000
9,David,77,90,230


So we sorted people by their salaries regardless of their department. The salary column is decreasing for each row and people in the same department are assigned the same salary. Department 14 gets 1000 for each row. We can also use ORDER BY() within the OVER() clause, but its functioning will be different. Below, we will remove the ORDER BY from the end and place it in the OVER() clause.

In [45]:
cursor = conn.execute(''' SELECT name, Department, Salary, SUM(Salary)
 OVER(PARTITION BY Department Order BY Salary DESC) as SUM_Salary FROM Employees''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'SUM_Salary'])

,name,Department,Salary,SUM_Salary
0,Mary-Anne,14,500,500
1,Anand,14,300,800
2,Michael,14,100,900
3,Elizabeth,14,50,1000
4,Kumar,14,50,1000
5,Carol,37,200,200
6,Joe,37,150,350
7,Zacary,59,600,600
8,John,59,130,860
9,Eric,59,130,860


At first peek, the salary column is no longer universally decreasing, and we see that we got a decresing order within each department. If you look at ('Mary-Anne', 14, 500, 500) row from above, you will see that the sum worked differently. In the "usual" Order By, Mary-Anne group salary sum was 1000 but the above shows that it is 500! Order By in OVER() clause orders by as intended per grouping, but the rows are presented in a summing total due to SUM() component of the window function. MAry-Anne has the highest salary (500) within department 14, and we started with her due to DESC expression. Then, it continues with second highest salary and gives a running total (500 + 300 = 800 For Anand, 800+100= 900 for Elizabeth and so on. If we look at the row with the least salary in the IT, we also see that the window sum is attached to that row (Kumar with salary 50 and window sum is 1000).

- Let's re-write again. Partition by defines the area to apply window function. Order by is then restricted to the specified window. If no order by is used, it reports the sum for each group. If we order by it, then, summing is done step by step per each row.

So far we worked with only SUM() function. Now, we will see other functions that are used in window functions.

**Row_number()**

Row_number provides the sequential order of the rows. If used with unspecified OVER(), it simply provides an increasing number for the entire data. This is useful when you want to get an= monotonically increasing ID column for each row.

In [47]:
# row numbers for the whole data
cursor = conn.execute(''' SELECT name, Department, Salary,
          ROW_NUMBER()  OVER() AS ROW_NUMBER FROM Employees''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'ROW_NUMBER'])

,name,Department,Salary,ROW_NUMBER
0,Michael,14,100,1
1,Anand,14,300,2
2,Carol,37,200,3
3,Joe,37,150,4
4,Mary-Anne,14,500,5
5,George,77,140,6
6,John,59,130,7
7,David,77,90,8
8,Zacary,59,600,9
9,Eric,59,130,10


We can get sequential increase for each group to make it more tailored to group analysis. In this case, the counting will reset when the new group appears, like so:

In [48]:
# row numbers per dept.
cursor = conn.execute(''' SELECT name, Department, Salary, 
                          ROW_NUMBER()  OVER(PARTITION BY Department) AS ROW_NUMBER 
                          FROM Employees''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'ROW_NUMBER'])

,name,Department,Salary,ROW_NUMBER
0,Michael,14,100,1
1,Anand,14,300,2
2,Mary-Anne,14,500,3
3,Elizabeth,14,50,4
4,Kumar,14,50,5
5,Carol,37,200,1
6,Joe,37,150,2
7,John,59,130,1
8,Zacary,59,600,2
9,Eric,59,130,3


If you noticed, we have seen again a transition from the entire data to the group, and to the individual row by moving from OVER() to OVER(Partition By). This was not the case for the aggreage function SUM(). This is the nature of Row_Number as it is inherently row-based. 

If we want to rank people based on their salary within each department from highest to minimum, we ORDER BY in descending order. In this case, ROW_NUMBER 1 will be highest earner in each department. ROW_NUMBER 2 will be the second highest and so on. 

In [49]:
# row numbers per dept.
cursor = conn.execute(''' SELECT name, Department, Salary, 
                          ROW_NUMBER()  OVER(PARTITION BY Department ORDER BY Salary DESC) ROW_NUMBER 
                          FROM Employees''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'ROW_NUMBER'])

,name,Department,Salary,ROW_NUMBER
0,Mary-Anne,14,500,1
1,Anand,14,300,2
2,Michael,14,100,3
3,Elizabeth,14,50,4
4,Kumar,14,50,5
5,Carol,37,200,1
6,Joe,37,150,2
7,Zacary,59,600,1
8,John,59,130,2
9,Eric,59,130,3


You may see the immediate use of this. You can answer questions like, "give me the 2nd highest salary earner in each department" etc. We do this by filtering the query on Row_Number=2. There is also Nth_Value function that can be used for this purpose, so you may want to check it out too.

In [50]:
# Second highest salaries for each department.
cursor = conn.execute(''' SELECT * FROM (SELECT name, Department, Salary, 
                          ROW_NUMBER() OVER(PARTITION BY Department ORDER BY Salary DESC) ROW_NUMBER 
                          FROM Employees)
 WHERE ROW_NUMBER=2 ''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'ROW_NUMBER'])

,name,Department,Salary,ROW_NUMBER
0,Anand,14,300,2
1,Joe,37,150,2
2,John,59,130,2
3,David,77,90,2


**Rank and Dense_Rank**

Rank() and Dense_Rank(), as their name suggests, does ranking in the predefined window. The difference between these two and ROW_NUMBER() is their sensitivity to the values they are ranking. ROW_NUMBER() does not allow ties, but these two functions recognize ties. However, Rank() and Dense_Rank() vary in how they rank after the tie. Let's see what we get when we use all three without using ORDER BY.

In [35]:
cursor = conn.execute(''' SELECT name, Department, Salary, 
ROW_NUMBER()  OVER(PARTITION BY Department) ROW_NUMBER,
RANK()  OVER(PARTITION BY Department) RANK,
DENSE_RANK()  OVER(PARTITION BY Department) DENSE_RANK
FROM Employees''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'ROW_NUMBER', 'RANK', 'DENSE_RANK'])

,name,Department,Salary,ROW_NUMBER,RANK,DENSE_RANK
0,Michael,14,100,1,1,1
1,Anand,14,300,2,1,1
2,Mary-Anne,14,500,3,1,1
3,Elizabeth,14,50,4,1,1
4,Kumar,14,50,5,1,1
5,Carol,37,200,1,1,1
6,Joe,37,150,2,1,1
7,John,59,130,1,1,1
8,Zacary,59,600,2,1,1
9,Eric,59,130,3,1,1


When used without ORDER BY, the query operated on the entire data, and we did not get a meaningful result for RANK() and DENSE_RANK(). Again, just like ROW_NUMBER(), RANK() and DENSE_RANK() are also row-based functions. Using ORDER BY is necessary in this case.


In [36]:
# Ordering by Salary
cursor = conn.execute(''' SELECT name, Department, Salary, 
ROW_NUMBER()  OVER(PARTITION BY Department ORDER BY Salary) ROW_NUMBER,
RANK()  OVER(PARTITION BY Department ORDER BY Salary) RANK,
DENSE_RANK()  OVER(PARTITION BY Department ORDER BY Salary) DENSE_RANK
FROM Employees''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'ROW_NUMBER', 'RANK', 'DENSE_RANK'])

,name,Department,Salary,ROW_NUMBER,RANK,DENSE_RANK
0,Elizabeth,14,50,1,1,1
1,Kumar,14,50,2,1,1
2,Michael,14,100,3,3,2
3,Anand,14,300,4,4,3
4,Mary-Anne,14,500,5,5,4
5,Joe,37,150,1,1,1
6,Carol,37,200,2,2,2
7,John,59,130,1,1,1
8,Eric,59,130,2,1,1
9,Zacary,59,600,3,3,2


Now, we got both group-level and row-level information. As can be seen in the second row with Michael, RANK() allows ties and skip the next ranking in line. Rank skipped rank number 2 and labeled Michael as 3rd. DENSE_RANK() too allows ties but it continues where the ranking is left off. Michael got 2nd place in the DENSE_RANK() results. The use of rank and dense_rank is decided based on your needs. If you are in some sports competition, you would not skip the third-place ranking when there are two second-place winners in the game.

**NTILE()**

NTILE() determines the percentile, quartile if a row is in. It returns the place for each of the rows in the partition. If there is no partition, it considers the entire data. This is useful for getting percentage level information, such as listing the top 25 percent earner, etc. Notice, we do not partition by the department below, therefore the query operates on the entire data.

In [51]:
# Percentile
cursor = conn.execute(''' SELECT name, Department, Salary, 
                          NTILE(4)  OVER(ORDER BY Salary) AS QUARTILE 
                          FROM Employees''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'QUARTILE'])

,name,Department,Salary,QUARTILE
0,Elizabeth,14,50,1
1,Kumar,14,50,1
2,David,77,90,1
3,Michael,14,100,2
4,John,59,130,2
5,Eric,59,130,2
6,George,77,140,3
7,Joe,37,150,3
8,Carol,37,200,3
9,Anand,14,300,4


Seems like Anand, Mary-Anne, and Zachary are the top 25% earners. Let's get quartiles for each group now by specifying the PARTITION BY() clause.

In [52]:
cursor = conn.execute(''' SELECT name, Department, Salary, 
                          NTILE(4)  OVER(PARTITION BY Department ORDER BY Salary) AS QUARTILE
                          FROM Employees''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'QUARTILE'])

,name,Department,Salary,QUARTILE
0,Elizabeth,14,50,1
1,Kumar,14,50,1
2,Michael,14,100,2
3,Anand,14,300,3
4,Mary-Anne,14,500,4
5,Joe,37,150,1
6,Carol,37,200,2
7,John,59,130,1
8,Eric,59,130,2
9,Zacary,59,600,3


**LEAD() and LAG()**

You may be familiar with lead and lag values in panel data analysis or time series. Just like we use them in those situations, SQL usage is also useful for comparing the value of the current row to that of the preceding or succeeding row. The first value lag gets NaN since there is no previous value, as is the case with the lead for the last value.

In [40]:
cursor = conn.execute(''' SELECT name, Department, Salary, 
LEAD(Salary, 1) OVER(PARTITION BY Department ORDER BY Salary) AS Next_Salary,
LAG(Salary, 1) OVER(PARTITION BY Department ORDER BY Salary) AS Prev_Salary  
FROM Employees''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'Next_Salary', 'Prev_Salary'])

,name,Department,Salary,Next_Salary,Prev_Salary
0,Elizabeth,14,50,50.0,NaN
1,Kumar,14,50,100.0,50.0
2,Michael,14,100,300.0,50.0
3,Anand,14,300,500.0,100.0
4,Mary-Anne,14,500,NaN,300.0
5,Joe,37,150,200.0,NaN
6,Carol,37,200,NaN,150.0
7,John,59,130,130.0,NaN
8,Eric,59,130,600.0,130.0
9,Zacary,59,600,NaN,130.0


Let's find out salary difference by subtracting previous person salary from the following one's salary.

In [42]:
cursor = conn.execute(''' SELECT name, Department, Salary, 
LAG(Salary, 1) OVER(PARTITION BY Department ORDER BY Salary) AS Prev_Salary,
Salary - LAG(Salary, 1) OVER(PARTITION BY Department ORDER BY Salary) AS Salary_Increase  
FROM Employees''')
data = []
for row in cursor:
  data.append(row)
pd.DataFrame(data, columns=['name', 'Department', 'Salary', 'Prev_Salary', 'Salary_Increase '])

,name,Department,Salary,Prev_Salary,Salary_Increase
0,Elizabeth,14,50,NaN,NaN
1,Kumar,14,50,50.0,0.0
2,Michael,14,100,50.0,50.0
3,Anand,14,300,100.0,200.0
4,Mary-Anne,14,500,300.0,200.0
5,Joe,37,150,NaN,NaN
6,Carol,37,200,150.0,50.0
7,John,59,130,NaN,NaN
8,Eric,59,130,130.0,0.0
9,Zacary,59,600,130.0,470.0


That concludes our discussion with window functions. There are more complex use cases for sure but knowing these basics will get you far for both in your daily work and your interviews. Hope this helped. Good luck!

# Appendix

In [ ]:

import pandas as pd
 
conn = sqlite3.connect('test.db')
 
conn.execute('''
CREATE TABLE Departments (
   Code INTEGER PRIMARY KEY NOT NULL,
   Name NVARCHAR NOT NULL ,
   Budget REAL NOT NULL 
 );''')
 
conn.commit()
 
print("Departments table created");
 
conn.execute('''
CREATE TABLE Employees (
   SSN INTEGER PRIMARY KEY NOT NULL,
   Name TEXT NOT NULL ,
   LastName VARCHAR NOT NULL ,
   Department INTEGER NOT NULL ,
   Salary INTEGER NOT NULL,
   CONSTRAINT fk_Departments_Code FOREIGN KEY(Department) 
   REFERENCES Departments(Code)
 );''')
 
conn.commit()
 
print("Employees table created")

# Inserting data to Departments table
conn.execute("INSERT INTO Departments(Code,Name,Budget) VALUES(14,'IT',65000);")
conn.execute("INSERT INTO Departments(Code,Name,Budget) VALUES(37,'Accounting',15000);")
conn.execute("INSERT INTO Departments(Code,Name,Budget) VALUES(59,'Human Resources',240000);")
conn.execute("INSERT INTO Departments(Code,Name,Budget) VALUES(77,'Research',55000);")
 
# Inserting data to Employees table
conn.execute("INSERT INTO Employees(SSN,Name,LastName,Department,Salary) VALUES('123234877','Michael','Rogers',14,100);")
conn.execute("INSERT INTO Employees(SSN,Name,LastName,Department,Salary) VALUES('152934485','Anand','Manikutty',14,300);")
conn.execute(" INSERT INTO Employees(SSN,Name,LastName,Department,Salary) VALUES('222364883','Carol','Smith',37,200); ")
conn.execute(" INSERT INTO Employees(SSN,Name,LastName,Department,Salary) VALUES('326587417','Joe','Stevens',37,150); ")
conn.execute(" INSERT INTO Employees(SSN,Name,LastName,Department,Salary) VALUES('332154719','Mary-Anne','Foster',14,500);")
conn.execute(" INSERT INTO Employees(SSN,Name,LastName,Department,Salary) VALUES('332569843','George','O''Donnell',77,140);")
conn.execute(" INSERT INTO Employees(SSN,Name,LastName,Department,Salary) VALUES('546523478','John','Doe',59,130);")
conn.execute("INSERT INTO Employees(SSN,Name,LastName,Department,Salary) VALUES('631231482','David','Smith',77,90);")
conn.execute(" INSERT INTO Employees(SSN,Name,LastName,Department,Salary) VALUES('654873219','Zacary','Efron',59,600);")
conn.execute(" INSERT INTO Employees(SSN,Name,LastName,Department,Salary) VALUES('745685214','Eric','Goldsmith',59,130);")
conn.execute(" INSERT INTO Employees(SSN,Name,LastName,Department,Salary) VALUES('845657245','Elizabeth','Doe',14,50);")
conn.execute(" INSERT INTO Employees(SSN,Name,LastName,Department,Salary) VALUES('845657246','Kumar','Swamy',14,50);")
conn.commit()